# 개인프로젝트
###  #01.프로젝트 개요

## 관심사
- 관심있는 브랜드의 남성 스포츠 용품에 대한 정보 수집 : 미즈노, 아디다스 


        미즈노 남성 스포츠용품 웹페이지 url : https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

        아디다스 남성 스포츠 용품 웹페이지 url : https://www.adidas.co.kr/men-performance?start=0


- 미즈노 크롤링 하나 데이터프레임+ 아디다스 크롤링 데이터 프레임 하나로  합치기 


In [13]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time


#### #02.웹 페이지 코드 수집 - 미즈노부터 ㄱㄱ 


미즈노는 각 페이지 별로 상품 수집해서 그 전에는 이미지링크 + 상품 상세보기 링크 + 상품이름 -> 링크를 타고 들어가자 
그다음에 정가, 할인가 + 리뷰 수 

In [14]:
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

## 접속객체 생성
session=requests.Session()

# 접속객체에 header정보 삽입
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,

})



In [15]:

## https://kor.mizuno.com/kr-kr/044/
## https://kor.mizuno.com/kr-kr/044/?start=0&sz=12

## 페이지마다 url ㄱ밧이 12씩 늘어남
mizuno_num =[]
for i in range(0,36) :
    mizuno_num.append(i*12)
print(mizuno_num)



[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156, 168, 180, 192, 204, 216, 228, 240, 252, 264, 276, 288, 300, 312, 324, 336, 348, 360, 372, 384, 396, 408, 420]


#### #03. 응답결과로부터 내용 추출하기

In [26]:
## 수집할 정보를 저장할 전체  list
## for 문이랑 url 리스트 돌리는 거 일단 나중에 ㄱㄱ

## 미즈노 빈 list
mizuno_result =[]
# ## 페이지 수 만큼 반복 
# ## 수집할 컨텐츠가 있는 웹페이지의 주소 
# url = "https://kor.mizuno.com/kr-kr/044/?start=0&sz=12" 
    
# ## 생성한 접속 객체를 활용해서 접속
# r = session.get(url)

# ## 에러 발생시 예외처리
# if r.status_code!=200 :
#     msg = "[%d Error] %s 에러가 발생함" % (r.status_code,r.reason)
#     raise Exception(msg)
    
# ## 인코딩 형식 지정
# r.encoding = 'utf-8'

# ## 전체 HTML 파일 
# soup = BeautifulSoup(r.text)

# mizuno_list = soup.select("#search-result-items")
# mizuno_list
#이미지링크 + 상품 상세보기 링크 + 상품이름
for number in mizuno_num :
    url = "https://kor.mizuno.com/kr-kr/044/?start=%d&sz=12" % number

    ## 생성한 접속 객체를 활용해서 접속
    r = session.get(url)

    ## 에러 발생시 예외처리
    if r.status_code!=200 :
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code,r.reason)
        raise Exception(msg)

    ## 인코딩 형식 지정
    r.encoding = 'utf-8'

    ## 전체 HTML 파일
    soup = BeautifulSoup(r.text)
    mizuno_list = soup.select("#search-result-items")
    for mizuno in mizuno_list :
        # print(mizuno)
        ## 상품이름 크롤링 & 상세보기 링크
        mizuno_n = mizuno.select(".product-tile > .product-name-and-pricing")
        for n in mizuno_n :
            mizuno_nameEl = n.select(".product-name> a ")
            mizuno_name = mizuno_nameEl[0].text.strip()
            # print(mizuno_name)

        ## 이미지 링크  mizuno_imageEl
        mizuno_i = mizuno.select(".product-tile > .tile-toparea")

        for i in mizuno_i :
            mizuno_imageEl = i.select(".product-image > .thumb-link > img")
            # print(mizuno_imageEl[0].attrs['src'])
            if mizuno_imageEl and "src" in mizuno_imageEl[0].attrs:
                mizuno_image = mizuno_imageEl[0].attrs['src']
            else : 
                mizuno_image =None
            # print(mizuno_image)
        
        ## 상품 상세 보기 링크  product-name-and-pricing
        mizuno_l = mizuno.select(".product-tile > .product-name-and-pricing ")
        for l in mizuno_l :
            mizuno_detailEl = l.select(".product-name > .name-link")
            mizuno_detail = mizuno_detailEl[0].attrs['href']
            # print(mizuno_detail)
        ## 링크 들어가서 상품상세정보 수집해야함 

        # ---------------------------------------------------
        
        #### 상세보기 수집 (시작) 

        # --------------mizuno_detail-------------------------------------

        chromedriver_autoinstaller.install()
        driver = webdriver.Chrome()
        driver.implicitly_wait(5)
        driver.get(mizuno_detail)
        time.sleep(1)
        


        

        detailSoup = BeautifulSoup(driver.page_source)
        # print(detailSoup)
        #      링크를 타고 들어가자 
        # 그다음에 정가, 할인금액,최종금액 + 리뷰 수
        detailEl = detailSoup.select(".pt_product-details")
        # print(detailEl)
        for detail in detailEl :
            standardPriceEl =detail.select("#product-content > .product-price > span ")
            # print(len(standardPriceEl))
            if len(standardPriceEl) ==1 :
                LastPrice = standardPriceEl[0].text.strip()
                standardPrice = LastPrice
                discountPrice = None
                # print(LastPrice)      
            elif len(standardPriceEl) == 3 :
                standardPrice = standardPriceEl[0].text.strip()
                LastPrice = standardPriceEl[1].text.strip()
                discountPrice = standardPriceEl[2].text.strip()
            reviewEl = detail.select(".naverReview > h2")
            # print(reviewEl[0].text.strip())
        result_dict = {'상품이름' :mizuno_name,"정가" : standardPrice,"할인금액":
                                discountPrice,"가격":LastPrice}
        mizuno_result.append(result_dict)

            

        



    



In [29]:
len(mizuno_result)

36